<a href="https://colab.research.google.com/github/marued/low-resource-machine-translation-team07/blob/master/notebooks/Semi_supervised_Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Semi-supervised  Neural Machine Translation
reproduce the result on our corpus, 
Project: https://github.com/artetxem/undreamt


In [5]:
import os
os.chdir("/home/ryan/projects/undreamt/data")
os.listdir()

['tokenizer.py',
 'train.lang1',
 'punctuation_remover.py',
 'unaligned.fr',
 'train.lang2',
 'unaligned.en',
 'evaluator.py']

In [7]:
!pip install sentencepiece
!pip install spacy

  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 10.6MB 126kB/s 
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/6c/f9/9a5658e2f56932e41eb264941f9a2cb7f3ce41a80cb36b2af6ab78e2f8af/catalogue-1.0.0-py2.py3-none-any.whl
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/21/e1/e4e7b754e6be3a79c400eb766fb34924a6d278c43bb828f94233e0124a21/wasabi-0.6.0-py3-none-any.whl
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 3.7MB 356kB/s 
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/86/85/40b8f66c2dd8f4fd9f09d59b22720cffecf1331e788b8a0cab5bafb353d1/plac-1.1.3-py2.py3-none-any.whl
  Cache entry deserialization failed, entry ignored


In [0]:
import sentencepiece as spm

## Train sentencpiece 

In [0]:
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

I will try different text pipeline for en and fr

- first config is en_uncased_nopunc, fr_cased_punc. This will not require a seperate auto punctuation model.

- second config is en_uncased_nopunc, fr_uncased_nopunc, This will require a seperate auto punctuation model.

In [0]:
!python tokenizer.py --input unaligned.en --output tokenized --lang en #--keep-case --keep-empty-lines
!python tokenizer.py --input unaligned.fr --output tokenized --lang fr --keep-case #--keep-empty-lines

In [15]:
!python punctuation_remover.py --input tokenized/unaligned.en --output fr_cased_punc

INFO:__main__:tokenizing file nopunc/unaligned.uncased.en
474000it [00:02, 167759.02it/s]
INFO:__main__:done - parsed 474000 lines and removed 993949 (punctuation) symbols


In [0]:
!cp tokenized/unaligned.fr fr_cased_punc/

In [19]:
os.chdir("/home/ryan/projects/undreamt/data/fr_cased_punc")
!wc -l train.lang1
!wc -l train.lang2

11000 train.lang1
11000 train.lang2


In [0]:
%%bash
head -10000 train.lang1 > sub_train.lang1
head -10000 train.lang2 > sub_train.lang2
tail -1000 train.lang1 > sub_test.lang1
tail -1000 train.lang2 > sub_test.lang2

In [0]:
# merge two corpus
! cat sub_train.lang1 unaligned.en > en_corpus.txt
! cat sub_train.lang2 unaligned.fr > fr_corpus.txt

In [25]:
# train sentencepiece model from `botchan.txt` and makes `m.model` and `m.vocab`
# `m.vocab` is just a reference. not used in the segmentation.
spm.SentencePieceTrainer.train('--input=en_corpus.txt --model_prefix=en_bpe --vocab_size=10000 --model_type=bpe')
spm.SentencePieceTrainer.train('--input=fr_corpus.txt --model_prefix=fr_bpe --vocab_size=10000 --model_type=bpe')


True

In [28]:
# Test
# makes segmenter instance and loads the model file
sp_en = spm.SentencePieceProcessor()
sp_en.load('en_bpe.model')

# encode: text => id
sentence="mr president ladies and gentlemen the financial perspective outlines the scope of the eu ’s activities over coming years as well as providing a framework for such activities and determining how effective they will be"
print(sp_en.encode_as_pieces(sentence))
print(sp_en.encode_as_ids(sentence))

encoded_sentence = sp_en.encode_as_pieces(sentence)
print(sp_en.decode_pieces(encoded_sentence))

['▁mr', '▁president', '▁ladies', '▁and', '▁gentlemen', '▁the', '▁financial', '▁perspective', '▁out', 'lines', '▁the', '▁scope', '▁of', '▁the', '▁eu', '▁’', 's', '▁activities', '▁over', '▁coming', '▁years', '▁as', '▁well', '▁as', '▁providing', '▁a', '▁framework', '▁for', '▁such', '▁activities', '▁and', '▁determining', '▁how', '▁effective', '▁they', '▁will', '▁be']
[192, 234, 1112, 39, 1114, 7, 700, 2856, 288, 3424, 7, 2924, 30, 7, 94, 308, 9965, 2075, 443, 1846, 519, 87, 536, 87, 3088, 5, 1273, 66, 474, 2075, 39, 8878, 259, 1157, 199, 138, 50]
mr president ladies and gentlemen the financial perspective outlines the scope of the eu ’s activities over coming years as well as providing a framework for such activities and determining how effective they will be


In [30]:
# Test
# makes segmenter instance and loads the model file
sp_fr = spm.SentencePieceProcessor()
sp_fr.load('fr_bpe.model')

# encode: text => id
sentence="Monsieur le Président , Mesdames et Messieurs , les perspectives financières esquissent la portée des activités de l ’ UE pour les années à venir , fournissent un cadre pour ces activités et déterminent leur efficacité ."
print(sp_fr.encode_as_pieces(sentence))
print(sp_fr.encode_as_ids(sentence))

['▁Monsieur', '▁le', '▁Président', '▁,', '▁Mesdames', '▁et', '▁Messieurs', '▁,', '▁les', '▁perspectives', '▁financières', '▁es', 'qu', 'issent', '▁la', '▁portée', '▁des', '▁activités', '▁de', '▁l', '▁’', '▁UE', '▁pour', '▁les', '▁années', '▁à', '▁venir', '▁,', '▁fournissent', '▁un', '▁cadre', '▁pour', '▁ces', '▁activités', '▁et', '▁détermin', 'ent', '▁leur', '▁efficacité', '▁.']
[352, 38, 351, 19, 1600, 42, 1574, 19, 49, 3207, 2372, 59, 15, 1362, 30, 4522, 53, 2193, 12, 5, 1438, 630, 82, 49, 818, 48, 2148, 19, 9758, 39, 891, 82, 250, 2193, 42, 3374, 18, 395, 2252, 17]


In [0]:
def bpe_encdoing(sentenceProcessor, filename):
    with open(filename, 'r') as f_in, open(f"{filename}.atok", 'w') as f_out:
        for line in f_in:
            f_out.writelines(" ".join(sentenceProcessor.encode_as_pieces(line)+ ['\n']))

In [0]:
sp_en = spm.SentencePieceProcessor()
sp_en.load('en_bpe.model')

bpe_encdoing(sp_en, "en_corpus.txt")

In [0]:
sp_fr = spm.SentencePieceProcessor()
sp_fr.load('fr_bpe.model')

bpe_encdoing(sp_fr, "fr_corpus.txt")

In [0]:
bpe_encdoing(sp_en, "sub_train.lang1")
bpe_encdoing(sp_en, "sub_test.lang1")

In [0]:
bpe_encdoing(sp_fr, "sub_train.lang2")
bpe_encdoing(sp_fr, "sub_test.lang2")

## Train word embedding

In [0]:
mkdir restult_dimension_300

In [44]:
!/home/ryan/projects/fastText/fasttext skipgram -input en_corpus.txt.atok -output restult_dimension_300/fasttext-en-300 -minn 2 -maxn 5 -dim 300

Read 10M words
Number of words:  9725
Number of labels: 0
Progress: 100.0% words/sec/thread:   25877 lr:  0.000000 avg.loss:  2.295455 ETA:   0h 0m 0s


In [45]:
!/home/ryan/projects/fastText/fasttext skipgram -input fr_corpus.txt.atok -output restult_dimension_300/fasttext-fr-300 -minn 2 -maxn 5 -dim 300

Read 13M words
Number of words:  9863
Number of labels: 0
Progress: 100.0% words/sec/thread:   24998 lr:  0.000000 avg.loss:  2.260107 ETA:   0h 0m 0s


Mapping two languange word embedding to same vector space

In [0]:
!pip install cupy

In [0]:
!python /home/ryan/projects/vecmap/map_embeddings.py --unsupervised restult_dimension_300/fasttext-en-300.vec restult_dimension_300/fasttext-fr-300.vec SRC_MAPPED.EMB TRG_MAPPED.EMB --cuda

## Train NMT model

In [50]:
import os
os.chdir("/home/ryan/projects/undreamt")
os.listdir('data/fr_cased_punc')

['en_bpe.model',
 'unaligned.cased.en',
 'sub_train.lang2',
 'unaligned.uncased.en',
 'train.lang1',
 'en_corpus.txt',
 'unaligned.cased.fr',
 'sub_test.lang2.atok',
 'sub_train.lang1',
 'sub_test.lang2',
 'sub_train.lang2.atok',
 'unaligned.fr',
 'sub_test.lang1',
 'sub_train.lang1.atok',
 'train.lang2',
 'fr_corpus.txt.atok',
 'en_corpus.txt.atok',
 'unaligned.en',
 'fr_corpus.txt',
 'en_bpe.vocab',
 'unaligned.uncased.fr',
 'fr_bpe.model',
 'sub_test.lang1.atok',
 'restult_dimension_300',
 'fr_bpe.vocab']

In [1]:
!pip install torch==0.3.1

In [0]:
!python train.py \
--src data/fr_cased_punc/en_corpus.txt.atok \
--trg data/fr_cased_punc/fr_corpus.txt.atok \
--src_embeddings data/fr_cased_punc/restult_dimension_300/SRC_MAPPED.EMB \
--trg_embeddings data/fr_cased_punc/restult_dimension_300/TRG_MAPPED.EMB \
--src2trg data/fr_cased_punc/sub_train.lang1.atok data/fr_cased_punc/sub_train.lang2.atok  \
--save en2fr-aligned_unaligned \
--save_interval 5000 \
--iterations 10000 \
--cuda

In [2]:
!python3 translate.py en2fr-aligned_unaligned.final.src2trg.pth < data/fr_cased_punc/sub_test.lang1.atok > data/fr_cased_punc/test_fr_pred.txt.atok

/home/ryan/projects/undreamt/undreamt/encoder.py:48: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  output, hidden = self.rnn(embeddings, hidden)


Get the BLEU score before decoding

In [9]:
 !perl multi-bleu.perl data/fr_cased_punc/sub_test.lang2.atok < data/fr_cased_punc/test_fr_pred.txt.atok 

BLEU = 11.11, 43.4/18.2/9.1/4.7 (BP=0.818, ratio=0.833, hyp_len=23055, ref_len=27674)


In [6]:
sp_fr = spm.SentencePieceProcessor()
sp_fr.load('data/fr_cased_punc/fr_bpe.model')

True

In [0]:
with open("data/fr_cased_punc/test_fr_pred.txt.atok", 'r') as f_in, open("data/fr_cased_punc/test_fr_pred.txt", 'w') as f_out:
    for line in f_in:
        f_out.writelines([sp_fr.decode_pieces(line.split()), '\n'])

After decoding the BPE text.

In [10]:
!perl multi-bleu.perl data/fr_cased_punc/sub_test.lang2 < data/fr_cased_punc/test_fr_pred.txt

BLEU = 11.97, 44.2/18.5/9.2/4.7 (BP=0.875, ratio=0.882, hyp_len=21865, ref_len=24780)


In [0]:
python3 translate.py lowresource_enfr.it2000.src2trg.dill < /home/ryan/projects/undreamt/data/fr_cased_punc/sub_test.lang1.atok > test_fr_pred.txt.atok